In [58]:
import numpy as np
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
import missingno as msno
from sklearn.impute import KNNImputer
from sklearn import tree
from sklearn.model_selection import train_test_split
sns.set_theme(style="whitegrid")
pd.options.display.max_columns = 200

In [115]:
data_produit=pd.read_csv('fr.openfoodfacts.org.products.csv',
                         sep = '\t', 
                         low_memory=False
                        )
data_produit

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_fr,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_fr,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_fr,ingredients_text,allergens,allergens_fr,traces,traces_tags,traces_fr,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_fr,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_fr,main_category,main_category_fr,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,caprylic-acid_100g,capric-acid_100g,lauric-acid_100g,myristic-acid_100g,palmitic-acid_100g,stearic-acid_100g,arachidic-acid_100g,behenic-acid_100g,lignoceric-acid_100g,cerotic-acid_100g,montanic-acid_100g,melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,alpha-linolenic-acid_100g,eicosapentaenoic-acid_100g,docosahexaenoic-acid_100g,omega-6-fat_100g,linoleic-acid_100g,arachidonic-acid_100g,gamma-linolenic-acid_100g,dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,oleic-acid_100g,elaidic-acid_100g,gondoic-acid_100g,mead-acid_100g,erucic-acid_100g,nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,sucrose_100g,glucose_100g,fructose_100g,lactose_100g,maltose_100g,maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,0000000003087,http://world-fr.openfoodfacts.org/produit/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,NaN,NaN,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:FR,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","A compléter,Informations nutritionnelles à com...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000004530,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,en:united-states,États-Unis,"Bananas, vegetable oil (coconut oil, corn oil ...",NaN,NaN,NaN,NaN,NaN,28 g (1 ONZ),NaN,0.0,[ bananas -> en:bananas ]

In [116]:
table_final=data_produit[['code','countries_fr','serving_size','nutrition_grade_fr',
                         'additives_n','pnns_groups_1','energy_100g','fat_100g','saturated-fat_100g',
                         'monounsaturated-fat_100g','polyunsaturated-fat_100g','cholesterol_100g','trans-fat_100g',
                         'carbohydrates_100g','sugars_100g','fiber_100g','proteins_100g',
                         'salt_100g']]
table_final.head(3)

,code,countries_fr,serving_size,nutrition_grade_fr,additives_n,pnns_groups_1,energy_100g,fat_100g,saturated-fat_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,cholesterol_100g,trans-fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g
0,0000000003087,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000004530,États-Unis,28 g (1 ONZ),d,0.0,NaN,2243.0,28.57,28.57,NaN,NaN,0.018,0.0,64.29,14.29,3.6,3.57,0.000
2,0000000004559,États-Unis,28 g (0.25 cup),b,0.0,NaN,1941.0,17.86,0.00,NaN,NaN,0.000,0.0,60.71,17.86,7.1,17.86,0.635


In [117]:
france_produit=table_final[table_final['countries_fr'].str.contains(pat='France',na=False)]

In [118]:
france_produit.describe(include='all')

,code,countries_fr,serving_size,nutrition_grade_fr,additives_n,pnns_groups_1,energy_100g,fat_100g,saturated-fat_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,cholesterol_100g,trans-fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g
count,98440,98440,26353,61415,53466.000000,64881,6.459300e+04,47642.000000,62375.000000,1186.000000,1205.000000,415.000000,386.000000,47211.000000,62515.000000,45723.000000,64318.000000,62574.000000
unique,98440,305,5699,5,NaN,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,0000000003087,France,30 g,d,NaN,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,94392,1737,16991,NaN,12498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,1.856619,NaN,1.171553e+03,13.332232,5.423696,18.574106,9.807383,0.041158,0.209285,27.759277,13.432792,2.559271,7.754531,1.160535
std,NaN,NaN,NaN,NaN,2.561405,NaN,1.283590e+04,16.926708,8.531083,21.960039,14.992141,0.535806,0.984866,27.413340,19.087618,4.634788,7.887373,4.309815
min,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.100000,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,0.000000,NaN,4.270000e+02,1.300000,0.300000,2.600000,1.500000,0.000000,0.000000,4.000000,1.000000,0.000000,1.800000,0.080000
50%,NaN,NaN,NaN,NaN,1.000000,NaN,1.035000e+03,6.800000,2.000000,9.100000,3.500000,0.000000,0.000000,14.500000,4.100000,1.380000,6.000000,0.558800
75%,NaN,NaN,NaN,NaN,3.000000,NaN,1.649000e+03,21.000000,7.400000,26.000000,11.000000,0.005440,0.115000,53.000000,17.800000,3.200000,11.000000,1.244600


In [119]:
france_produit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98440 entries, 0 to 320770
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   code                      98440 non-null  object 
 1   countries_fr              98440 non-null  object 
 2   serving_size              26353 non-null  object 
 3   nutrition_grade_fr        61415 non-null  object 
 4   additives_n               53466 non-null  float64
 5   pnns_groups_1             64881 non-null  object 
 6   energy_100g               64593 non-null  float64
 7   fat_100g                  47642 non-null  float64
 8   saturated-fat_100g        62375 non-null  float64
 9   monounsaturated-fat_100g  1186 non-null   float64
 10  polyunsaturated-fat_100g  1205 non-null   float64
 11  cholesterol_100g          415 non-null    float64
 12  trans-fat_100g            386 non-null    float64
 13  carbohydrates_100g        47211 non-null  float64
 14  sugar

In [120]:
france_produit=france_produit.dropna(subset=['code'])
france_produit[france_produit["code"].isna()]

,code,countries_fr,serving_size,nutrition_grade_fr,additives_n,pnns_groups_1,energy_100g,fat_100g,saturated-fat_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,cholesterol_100g,trans-fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g


In [121]:
france_produit=france_produit[france_produit.isnull().sum(axis = 1)<12]
france_produit.count()

code                        64516
countries_fr                64516
serving_size                25502
nutrition_grade_fr          61334
additives_n                 43757
pnns_groups_1               50269
energy_100g                 64455
fat_100g                    47229
saturated-fat_100g          62374
monounsaturated-fat_100g     1186
polyunsaturated-fat_100g     1205
cholesterol_100g              415
trans-fat_100g                386
carbohydrates_100g          47179
sugars_100g                 62439
fiber_100g                  45720
proteins_100g               64289
salt_100g                   62472
dtype: int64

In [122]:
remplacer1=['salt_100g','fiber_100g','sugars_100g','saturated-fat_100g','monounsaturated-fat_100g','polyunsaturated-fat_100g',
           'cholesterol_100g','trans-fat_100g','proteins_100g']
for i in remplacer1:
    france_produit.loc[france_produit[i]>100,i]=100

In [123]:
for i in remplacer1:
    france_produit[i].fillna(0,inplace=True)

In [124]:
france_produit['carbohydrates_100g'].fillna(france_produit['sugars_100g'],inplace=True)

In [125]:
france_produit['fat_100g'].fillna((france_produit['saturated-fat_100g']+france_produit['monounsaturated-fat_100g']+france_produit['polyunsaturated-fat_100g']),inplace=True)

In [126]:
remplacer2=['fat_100g','carbohydrates_100g']
for i in remplacer2:
    france_produit.loc[france_produit[i]>100,i]=np.nan
    france_produit=france_produit.dropna(subset=[i])

In [127]:
france_produit[france_produit['energy_100g'].isnull()]

,code,countries_fr,serving_size,nutrition_grade_fr,additives_n,pnns_groups_1,energy_100g,fat_100g,saturated-fat_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,cholesterol_100g,trans-fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g
109304,0097339000061,"France,États-Unis",5 g (1 tsp),NaN,NaN,NaN,NaN,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,3.2512
140901,0748927028669,"France,États-Unis",30.4 g,NaN,NaN,unknown,NaN,1.00,0.00,0.0,0.0,0.0,0.0,3.0,0.0,0.00,24.0,0.0000
175511,2000000009434,France,NaN,NaN,1.0,unknown,NaN,1.50,0.00,0.0,0.0,0.0,0.0,82.0,0.0,0.00,7.5,0.0000
176303,2000000035266,France,NaN,NaN,0.0,Beverages,NaN,0.01,0.01,0.0,0.0,0.0,0.0,0.5,0.5,0.01,0.5,0.0100
184441,26008059,France,NaN,NaN,4.0,Fat and sauces,NaN,70.00,17.00,24.0,28.0,0.0,1.0,0.3,0.0,0.00,0.2,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304953,8032697890222,France,NaN,NaN,0.0,Sugary snacks,NaN,28.70,0.00,0.0,0.0,0.0,0.0,52.4,0.0,0.00,12.4,0.0000
313616,8710873996332,France,NaN,NaN,NaN,unknown,NaN,9.00,4.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,4.0,0.5715
314929,8713600281169,France,195 g,NaN,NaN,Sugary snacks,NaN,1.50,1.40,0.0,0.0,0.0,0.0,88.0,86.0,0.00,0.0,0.0200
316935,8850987123641,"France,Suisse,États-Unis",NaN,NaN,6.0,Composite foods,NaN,2.60,1.40,0.0,0.0,0.0,0.0,6.4,0.7,0.00,1.2,0.8000


In [128]:
#france_produit.loc[france_produit['energy_100g']>3771,'energy_100g']=np.nan
france_produit.loc[abs(france_produit['energy_100g']-(4*(france_produit['carbohydrates_100g']+france_produit['proteins_100g'])+9*france_produit['fat_100g'])*4.187)>42,'energy_100g']=np.nan
#france_produit[france_produit['energy_100g'].isnull()]
france_produit['energy_100g'].fillna((4*(france_produit['carbohydrates_100g']+france_produit['proteins_100g'])+9*france_produit['fat_100g'])*4.187,inplace=True)   

In [129]:
france_produit.loc[france_produit['energy_100g']>3771,'energy_100g']=np.nan
france_produit=france_produit.dropna(subset=['energy_100g'])


In [130]:
france_produit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64478 entries, 106 to 320763
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   code                      64478 non-null  object 
 1   countries_fr              64478 non-null  object 
 2   serving_size              25490 non-null  object 
 3   nutrition_grade_fr        61298 non-null  object 
 4   additives_n               43738 non-null  float64
 5   pnns_groups_1             50243 non-null  object 
 6   energy_100g               64478 non-null  float64
 7   fat_100g                  64478 non-null  float64
 8   saturated-fat_100g        64478 non-null  float64
 9   monounsaturated-fat_100g  64478 non-null  float64
 10  polyunsaturated-fat_100g  64478 non-null  float64
 11  cholesterol_100g          64478 non-null  float64
 12  trans-fat_100g            64478 non-null  float64
 13  carbohydrates_100g        64478 non-null  float64
 14  sug

In [131]:
france_produit.loc[france_produit['pnns_groups_1']=='unknown','pnns_groups_1']=np.nan
france_produit.loc[france_produit['pnns_groups_1']=='fruits-and-vegetables','pnns_groups_1']='Fruits and vegetables'
france_produit.loc[france_produit['pnns_groups_1']=='sugary-snacks','pnns_groups_1']='Sugary snacks'
france_produit.loc[france_produit['pnns_groups_1']=='cereals-and-potatoes','pnns_groups_1']='Cereals and potatoes'
france_produit['pnns_groups_1'].unique()

array([nan, 'Beverages', 'Sugary snacks', 'Composite foods',
       'Fruits and vegetables', 'Milk and dairy products',
       'Fat and sauces', 'Salty snacks', 'Cereals and potatoes',
       'Fish Meat Eggs'], dtype=object)

In [132]:
france_produit['additives_n'] = france_produit['additives_n'].fillna(france_produit.groupby('pnns_groups_1')['additives_n'].transform('mean'))

In [133]:
fr_pro=france_produit.drop(columns=['code', 'countries_fr','serving_size','nutrition_grade_fr','additives_n'])

In [134]:
X_pre=fr_pro[fr_pro['pnns_groups_1'].isnull()].drop(columns=['pnns_groups_1'])
X=fr_pro[fr_pro['pnns_groups_1'].notnull()].drop(columns=['pnns_groups_1'])
y=fr_pro[fr_pro['pnns_groups_1'].notnull()][['pnns_groups_1']]
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

In [135]:
y_train.head()

,pnns_groups_1
217226,Composite foods
188263,Fruits and vegetables
200842,Beverages
210037,Cereals and potatoes
179084,Sugary snacks


In [136]:
clf = tree.DecisionTreeClassifier(random_state=0, max_depth=20)
clf = clf.fit(X_train, y_train)

In [137]:
clf.score(X_test, y_test)

0.8671617161716172

In [138]:
y_pre=clf.predict(X_pre)

In [139]:
france_produit.loc[france_produit['pnns_groups_1'].isnull(),'pnns_groups_1']=y_pre

In [140]:
france_produit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64478 entries, 106 to 320763
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   code                      64478 non-null  object 
 1   countries_fr              64478 non-null  object 
 2   serving_size              25490 non-null  object 
 3   nutrition_grade_fr        61298 non-null  object 
 4   additives_n               48196 non-null  float64
 5   pnns_groups_1             64478 non-null  object 
 6   energy_100g               64478 non-null  float64
 7   fat_100g                  64478 non-null  float64
 8   saturated-fat_100g        64478 non-null  float64
 9   monounsaturated-fat_100g  64478 non-null  float64
 10  polyunsaturated-fat_100g  64478 non-null  float64
 11  cholesterol_100g          64478 non-null  float64
 12  trans-fat_100g            64478 non-null  float64
 13  carbohydrates_100g        64478 non-null  float64
 14  sug

In [141]:
fr_pro=france_produit.drop(columns=['code', 'countries_fr','serving_size','additives_n','pnns_groups_1'])
X_pre=fr_pro[fr_pro['nutrition_grade_fr'].isnull()].drop(columns=['nutrition_grade_fr'])
X=fr_pro[fr_pro['nutrition_grade_fr'].notnull()].drop(columns=['nutrition_grade_fr'])
y=fr_pro[fr_pro['nutrition_grade_fr'].notnull()][['nutrition_grade_fr']]
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

In [142]:
clf = tree.DecisionTreeClassifier(random_state=0,max_depth=19)
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.8411092985318107

In [143]:
y_pre=clf.predict(X_pre)

In [144]:

france_produit.loc[france_produit['nutrition_grade_fr'].isnull(),'nutrition_grade_fr']=y_pre

In [145]:
france_produit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64478 entries, 106 to 320763
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   code                      64478 non-null  object 
 1   countries_fr              64478 non-null  object 
 2   serving_size              25490 non-null  object 
 3   nutrition_grade_fr        64478 non-null  object 
 4   additives_n               48196 non-null  float64
 5   pnns_groups_1             64478 non-null  object 
 6   energy_100g               64478 non-null  float64
 7   fat_100g                  64478 non-null  float64
 8   saturated-fat_100g        64478 non-null  float64
 9   monounsaturated-fat_100g  64478 non-null  float64
 10  polyunsaturated-fat_100g  64478 non-null  float64
 11  cholesterol_100g          64478 non-null  float64
 12  trans-fat_100g            64478 non-null  float64
 13  carbohydrates_100g        64478 non-null  float64
 14  sug

In [146]:
france_produit['additives_n'] = france_produit['additives_n'].fillna(france_produit.groupby('pnns_groups_1')['additives_n'].transform('mean'))

In [150]:
france_produit=france_produit.round({"additives_n":0}) 
france_produit

,code,countries_fr,serving_size,nutrition_grade_fr,additives_n,pnns_groups_1,energy_100g,fat_100g,saturated-fat_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,cholesterol_100g,trans-fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g
106,0000000036252,"France,États-Unis",40 g (0.25 cup),e,1.0,Sugary snacks,1967.89000,20.00,12.50,0.0,0.0,0.012,0.0,70.00,57.50,2.5,2.50,0.09652
138,0000000039529,"France,États-Unis",48 g (0.25 cup),a,3.0,Sugary snacks,1552.74895,4.17,0.00,0.0,0.0,0.000,0.0,77.08,0.00,6.2,6.25,0.00000
190,0000010187319,France,NaN,d,2.0,Beverages,1508.99480,0.80,0.80,0.0,0.0,0.000,0.0,87.70,87.70,0.9,0.60,0.01000
191,0000010207260,France,NaN,d,3.0,Sugary snacks,1110.81110,2.90,2.90,0.0,0.0,0.000,0.0,50.30,50.30,3.9,9.50,0.00300
226,0000040608754,France,150ml,e,2.0,Beverages,177.00000,0.00,0.00,0.0,0.0,0.000,0.0,10.40,10.40,0.0,0.00,0.02540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320740,9782211109758,France,NaN,d,2.0,Milk and dairy products,773.75760,12.00,12.00,0.0,0.0,0.000,0.0,10.50,10.50,0.0,8.70,0.29000
320741,9782401029101,France,NaN,b,2.0,Fish Meat Eggs,221.91100,1.00,1.00,0.0,0.0,0.000,0.0,1.00,1.00,10.0,10.00,10.00000
320751,9847548283004,France,NaN,a,1.0,Cereals and potatoes,1575.98680,2.80,0.60,0.0,0.0,0.000,0.0,74.80,2.60,5.9,13.00,0.68000
320757,9900000000233,France,NaN,b,1.0,Cereals and potatoes,561.09987,3.73,3.73,0.0,0.0,0.000,0.0,3.89,3.89,12.2,21.22,0.10000


In [151]:
france_produit.to_csv("france_produit.csv",index=False) 